# Building Models in PyMC3

Bayesian inference begins with specification of a probability model relating unknown variables to data. PyMC3 provides the basic building blocks for Bayesian probability models: stochastic random variables, deterministic variables, and factor potentials. 

A **stochastic random variable** is a factor whose value is not completely determined by its parents, while the value of a **deterministic random variable** is entirely determined by its parents. Most models can be constructed using only these two variable types. The third quantity, the **factor potential**, is *not* a variable but simply a
log-likelihood term or constraint that is added to the joint log-probability to modify it. 

## The FreeRV class

A stochastic variable is represented in PyMC3 by a `FreeRV` class. This structure adds functionality to Theano's `TensorVariable` class, by mixing in the PyMC `Factor` class. A `Factor` is used whenever a variable contributes a log-probability term to a model. Hence, you know a variable is a subclass of `Factor` whenever it has a `logp` method, as we saw in the previous section.

A `FreeRV` object has several important attributes:

`dshape`
:   The variable's shape.

`dsize`
:   The overall size of the variable.

`distribution`
:   The probability density or mass function that describes the distribution of the variable's values.

`logp`
:   The log-probability of the variable's current value given the values
    of its parents.

`init_value`
:   The initial value of the variable, used by many algorithms as a starting point for model fitting.

`model`
:   The PyMC model to which the variable belongs.


### Creation of stochastic random variables

There are two ways to create stochastic random variables (`FreeRV` objects), which we will call the **automatic**, and **manual** interfaces.

#### Automatic

Stochastic random variables with standard distributions provided by PyMC3 can be created in a single line using special subclasses of the `Distribution` class. For example, as we have seen, the uniformly-distributed discrete variable $switchpoint$ in the coal mining disasters model is created using the automatic interface as follows:

In [3]:
import pymc3 as pm

with pm.Model() as disaster_model:

    switchpoint = pm.DiscreteUniform('switchpoint', lower=0, upper=110)

Similarly, the rate parameters can automatically be given exponential priors:

In [5]:
with disaster_model:
    early_mean = pm.Exponential('early_mean', lam=1)
    late_mean = pm.Exponential('late_mean', lam=1)

Applied log-transform to early_mean and added transformed early_mean_log to model.
Applied log-transform to late_mean and added transformed late_mean_log to model.


PyMC includes most of the probability density functions (for continuous variables) and probability mass functions (for discrete variables) used in probabilistic modeling. Continuous variables are represented by a specialized subclass of `Distribution` called `Continuous` and discrete variables by the `Discrete` subclass.

The main difference between these two sublcasses are in the `dtype` attribute (`int64` for `Discrete` and `float64` for `Continuous`) and the `defaults` attribute, which determines which summary statistic to use for initial values when one is not specified ('mode' for `Discrete` and 'median', 'mean', and 'mode' for `Continuous`).

In [34]:
switchpoint.distribution.defaults

['mode']

As we previewed in the introduction, `Distribution` has a class method `dist` that returns a probability distribution of that type, without being wrapped in a PyMC random variable object. Sometimes we wish to use a particular statistical distribution, without using it as a variable in a model; for example, to generate random numbers from the distribution. This class method allows that.

In [9]:
pm.Exponential.dist(1)

#### Manual

The uniformly-distributed discrete stochastic variable `switchpoint` in the disasters model could alternatively be created from a function that computes its log-probability as follows:

In [65]:
import numpy as np

with pm.Model():
    
    def uniform_logp(value, lower=0, upper=111):
        """The switchpoint for the rate of disaster occurrence."""
        return pm.switch((value > upper) | (value < lower), -np.inf, -np.log(upper - lower + 1))

    switchpoint = pm.DensityDist('switchpoint', logp=uniform_logp, dtype='int64')

In [52]:
switchpoint.logp({'switchpoint':4})

array(-4.718498871295094)

In [59]:
switchpoint.logp({'switchpoint': 44})

array(-4.718498871295094)

In [54]:
switchpoint.logp({'switchpoint':-1})

array(-inf)

A couple of things to notice: while the function specified for the `logp` argument can be an arbitrary Python function, it must use Theano operators in its body. This is because one or more of the arguments passed to the function may be `TensorVariables`, and they must be supported. Also, we passed the value to be evaluated by the `logp` function as a dictionary, rather than as a plain integer. By convention, values in PyMC3 are passed around as a data structure called a `Point`. Points in parameter space are represented by dictionaries with parameter names as they keys and the value of the parameters as the values.

To emphasize, the Python function passed to `DensityDist` should compute the *log*-density or *log*-probability of the variable. That is why the return value in the example above is `-log(upper-lower+1)` rather than `1/(upper-lower+1)`.

## The ObservedRV Class

Stochastic random variables whose values are observed (*i.e.* data likelihoods) are represented by a different class than unobserved random variables. A `ObservedRV` object is instantiated any time a stochastic variable is specified with data passed as the `observed` argument. 

Otherwise, observed stochastic random variables are created via the same interfaces as unobserved: **automatic** or **manual**. As an example of an automatic instantiation, consider a Poisson data likelihood :

In [75]:
with disaster_model:
    
    disasters = pm.Poisson('disasters', mu=3, observed=[3,4,1,2,0,2,2])

We have already seen manual instantiation, from the melanoma survial model where the exponential survival likelihood was implemented manually:

```python
def logp(failure, value):
    return (failure * log(lam) - lam * value).sum()

x = DensityDist('x', logp, observed={'failure':failure, 'value':t})
```

Notice in this example that there are two vetors observed data for the likelihood `x`, passed as a dictionary.

An important responsibility of `ObservedRV` is to automatically handle missing values in the data, when they are present (absent?). More on this later.

## Deterministic Variables

A deterministic variable is one whose values are *completely determined* by the values of their parents. For example, in our disasters model, `rate` is a deterministic variable.

In [67]:
with disaster_model:
    
    rate = pm.Deterministic('rate', pm.switch(switchpoint >= np.arange(112), early_mean, late_mean))

so `rate`'s value can be computed exactly from the values of its parents `early_mean`, `late_mean` and `switchpoint`.

There are two types of deterministic variables in PyMC3

#### Anonymous deterministic variables

The easiest way to create a deterministic variable is to operate on or transform one or more variables in a model directly. For example, the simplest way to specify the `rate` variable above is as follows:

In [69]:
with disaster_model:
    
    rate = pm.switch(switchpoint >= np.arange(112), early_mean, late_mean)

Or, let's say we wanted to use the mean of the `early_mean` and `late_mean` variables somehere in our model:

In [70]:
with disaster_model:
    
    mean_of_means = (early_mean + late_mean)/2

These are called *anonymous* variables because we did not wrap it with a call to `Determinstic`, which gives it a name as its first argument. We simply specified the variable as a Python (or, Theano) expression. This is therefore the simplest way to construct a determinstic variable. The only caveat is that the values generated by anonymous determinstics at every iteration of a MCMC algorithm, for example, are not recorded to the resulting trace. So, this approach is only appropriate for intermediate values in your model that you do not wish to obtain posterior estimates for, alongside the other variables in the model.

#### Named deterministic variables

To ensure that deterministic variables' values are accumulated during sampling, they should be instantiated using the **named deterministic** interface; this uses the `Deterministic` function to create the variable. Two things happen when a variable is created this way:

1. The variable is given a name (passed as the first argument)
2. The variable is appended to the model's list of random variables, which ensures that its values are tallied.


In [ ]:
def rate_eval(switchpoint=switchpoint, early_mean=early_mean, late_mean=late_mean):
    value = np.zeros(111)
    value[:switchpoint] = early_mean
    value[switchpoint:] = late_mean
    return value

rate = pm.Deterministic(eval = rate_eval,
                  name = 'rate',
                  parents = {'switchpoint': switchpoint, 
                          'early_mean': early_mean, 
                          'late_mean': late_mean},
                  doc = 'The rate of disaster occurrence.',
                  trace = True,
                  verbose = 0,
                  dtype=float,
                  plot=False,
                  cache_depth = 2)

## Factor Potentials

For some applications, we want to be able to modify the joint density by incorporating terms that don't correspond to probabilities of variables conditional on parents, for example:

$$p(x_0, x_2, \ldots x_{N-1}) \propto \prod_{i=0}^{N-2} \psi_i(x_i, x_{i+1})$$

In other cases we may want to add probability terms to existing models. For example, suppose we want to constrain the difference between the early and late means in the disaster model to be less than 1, so that the joint density becomes: 

$$p(y,\tau,\lambda_1,\lambda_2) \propto p(y|\tau,\lambda_1,\lambda_2) p(\tau) p(\lambda_1) p(\lambda_2) I(|\lambda_2-\lambda_1| \lt 1)$$

We call such log-probability terms **factor potentials** (Jordan 2004). Bayesian
hierarchical notation doesn't accomodate these potentials. 

### Creation of Potentials

A potential can be created via the `Potential` function, in a way very similar to `Deterministic`'s named interface:

In [72]:
with disaster_model:
    
    rate_constraint = pm.Potential('rate_constraint', pm.switch(pm.abs_(early_mean-late_mean)>1, -np.inf, 0))

The function takes just a `name` as its first argument and an expression returning the appropriate log-probability as the second argument.

A common use of a factor potential is to represent an observed likelihood, where the observations are partly a function of model variables. In the contrived example below, we are representing the error in a linear regression model as a zero-mean normal random variable. Thus, the "data" in this scenario is the residual, which is a function both of the data and the regression parameters. 

In [78]:
y = np.array([15, 10, 16, 11, 9, 11, 10, 18, 11])
x = np.array([1, 2, 4, 5, 6, 8, 19, 18, 12])

with pm.Model() as arma_model:

    sigma = pm.HalfCauchy('sigma', 5)
    beta = pm.Normal('beta', 0, sd=2)
    mu = pm.Normal('mu', 0, sd=10)

    err = y - (mu + beta*x)
                  
    like = pm.Potential('like', pm.Normal.dist(0, sd=sigma).logp(err))

Applied log-transform to sigma and added transformed sigma_log to model.


This parameterization would not be compatible with an observed stochastic, because the `err` term would become fixed in the likelihood and not be allowed to change during sampling.

## Exercise: Bioassay model

Gelman et al. (2003) present an example of an acute toxicity test, commonly performed on animals to estimate the toxicity of various compounds.

In this dataset `log_dose` includes 4 levels of dosage, on the log scale, each administered to 5 rats during the experiment. The response variable is death, the number of positive responses to the dosage.

The number of deaths can be modeled as a binomial response, with the probability of death being a linear function of dose:

$$\begin{aligned}
y_i &\sim \text{Bin}(n_i, p_i) \\
\text{logit}(p_i) &= a + b x_i
\end{aligned}$$

The common statistic of interest in such experiments is the LD50, the dosage at which the probability of death is 50%.

Specify this model in PyMC:

In [ ]:
# Log dose in each group
log_dose = [-.86, -.3, -.05, .73]

# Sample size in each group
n = 5

# Outcomes
deaths = [0, 1, 3, 5]

In [ ]:
## Write your answer here

## Sampling with MCMC

PyMC provides three objects that fit models:

- `MCMC`, which coordinates Markov chain Monte Carlo algorithms. The actual work of updating stochastic variables conditional on the rest of the model is done by `StepMethod` objects.

- `MAP`, which computes maximum *a posteriori* estimates.

- `NormApprox`, the joint distribution of all stochastic variables in a model is approximated as normal using local information at the maximum *a posteriori* estimate.

All three objects are subclasses of `Model`, which is PyMC's base class
for fitting methods. `MCMC` and `NormApprox`, both of which can produce
samples from the posterior, are subclasses of `Sampler`, which is PyMC's
base class for Monte Carlo fitting methods. `Sampler` provides a generic
sampling loop method and database support for storing large sets of
joint samples. These base classes implement some basic methods that are
inherited by the three implemented fitting methods, so they are
documented at the end of this section.


### MCMC

The `MCMC` class implements PyMC's core business: producing Markov chain Monte Carlo samples for
a model's variables. Its primary job is to create and coordinate a collection of 'step
methods', each of which is responsible for updating one or more
variables. 

`MCMC` provides the following useful methods:

`sample(iter, burn, thin, tune_interval, tune_throughout, save_interval, ...)`
:   Runs the MCMC algorithm and produces the traces. The `iter` argument
controls the total number of MCMC iterations. No tallying will be
done during the first `burn` iterations; these samples will be
forgotten. After this burn-in period, tallying will be done each
`thin` iterations. Tuning will be done each `tune_interval`
iterations. If `tune_throughout=False`, no more tuning will be done
after the burnin period. The model state will be saved every
`save_interval` iterations, if given.

`isample(iter, burn, thin, tune_interval, tune_throughout, save_interval, ...)`
:   An interactive version of `sample`. The sampling loop may be paused
at any time, returning control to the user.

`use_step_method(method, *args, **kwargs)`:
:   Creates an instance of step method class `method` to handle some
stochastic variables. The extra arguments are passed to the `init`
method of `method`. Assigning a step method to a variable manually
will prevent the `MCMC` instance from automatically assigning one.
However, you may handle a variable with multiple step methods.

`stats()`:
:   Generate summary statistics for all nodes in the model.

The sampler's MCMC algorithms can be accessed via the `step_method_dict`
attribute. `M.step_method_dict[x]` returns a list of the step methods
`M` will use to handle the stochastic variable `x`.

After sampling, the information tallied by `M` can be queried via
`M.db.trace_names`. In addition to the values of variables, tuning
information for adaptive step methods is generally tallied. These
‘traces’ can be plotted to verify that tuning has in fact terminated. After sampling ends you can retrieve the trace as
`M.trace[’var_name’]`.

We can instantiate a MCMC sampler for the bioassay example as follows:

In [ ]:
M = pm.MCMC(gelman_bioassay, db='sqlite')

#### Sampling methods

Though finding the MAP is a fast and easy way of obtaining estimates of the unknown model parameters, it is limited because there is no associated estimate of uncertainty produced with the MAP estimates. Instead, a simulation-based approach such as Markov chain Monte Carlo (MCMC) can be used to obtain a Markov chain of values that, given the satisfaction of certain conditions, are indistinguishable from samples from the posterior distribution. 

To conduct MCMC sampling to generate posterior samples in PyMC3, we specify a **step method** object that corresponds to a particular MCMC algorithm, such as Metropolis, Slice sampling, or the No-U-Turn Sampler (NUTS). PyMC3's `step_methods` submodule contains the following samplers: `NUTS`, `Metropolis`, `Slice`, `HamiltonianMC`, and `BinaryMetropolis`. These step methods can be assigned manually, or assigned automatically by PyMC3. Auto-assignment is based on the attributes of each variable in the model. In general:

* Binary variables will be assigned to `BinaryMetropolis`
* Discrete variables will be assigned to `Metropolis`
* Continuous variables will be assigned to `NUTS`

Auto-assignment can be overriden for any subset of variables by specifying them manually prior to sampling.

## Step methods

Step method objects handle individual stochastic variables, or sometimes groups 
of them. They are responsible for making the variables they handle take single 
MCMC steps conditional on the rest of the model. Each subclass of 
``StepMethod`` implements a method called ``step()``, which is called by 
``MCMC``. Step methods with adaptive tuning parameters can optionally implement 
a method called ``tune()``, which causes them to assess performance (based on 
the acceptance rates of proposed values for the variable) so far and adjust.

The major subclasses of ``StepMethod`` are ``Metropolis`` and
``AdaptiveMetropolis``. PyMC provides several flavors of the 
basic Metropolis steps.

### Metropolis

``Metropolis`` and subclasses implement Metropolis-Hastings steps. To tell an 
``MCMC`` object :math:`M` to handle a variable :math:`x` with a Metropolis step 
method, you might do the following:

In [ ]:
M.use_step_method(pm.Metropolis, M.alpha, proposal_sd=1., proposal_distribution='Normal')

`Metropolis` itself handles float-valued variables, and subclasses
`DiscreteMetropolis` and `BinaryMetropolis` handle integer- and
boolean-valued variables, respectively.

`Metropolis`' `__init__` method takes the following arguments:

`stochastic`
:   The variable to handle.

`proposal_sd`
:   A float or array of floats. This sets the proposal standard deviation if the proposal distribution is normal.

`scale`
:   A float, defaulting to 1. If the `scale` argument is provided but not `proposal_sd`, `proposal_sd` is computed as follows:

```python
if all(self.stochastic.value != 0.):
    self.proposal_sd = (ones(shape(self.stochastic.value)) * 
                   abs(self.stochastic.value) * scale)
else:
    self.proposal_sd = ones(shape(self.stochastic.value)) * scale
```

`proposal_distribution`
:   A string indicating which distribution should be used for proposals.
Current options are `'Normal'` and `'Prior'`. If
`proposal_distribution=None`, the proposal distribution is chosen
automatically. It is set to `'Prior'` if the variable has no
children and has a random method, and to `'Normal'` otherwise.

Alhough the `proposal_sd` attribute is fixed at creation, Metropolis
step methods adjust their initial proposal standard deviations using an
attribute called `adaptive_scale_factor`. During tuning, the
acceptance ratio of the step method is examined, and this scale factor
is updated accordingly. If the proposal distribution is normal,
proposals will have standard deviation
`self.proposal_sd * self.adaptive_scale_factor`.

By default, tuning will continue throughout the sampling loop, even
after the burnin period is over. This can be changed via the
`tune_throughout` argument to `MCMC.sample`. If an adaptive step
method's `tally` flag is set (the default for `Metropolis`), a trace of
its tuning parameters will be kept. If you allow tuning to continue
throughout the sampling loop, it is important to verify that the
'Diminishing Tuning' condition of [Roberts and Rosenthal (2007)](http://projecteuclid.org/DPubS?service=UI&version=1.0&verb=Display&handle=euclid.jap/1183667414) is satisfied: the
amount of tuning should decrease to zero, or tuning should become very
infrequent.

If a Metropolis step method handles an array-valued variable, it
proposes all elements independently but simultaneously. That is, it
decides whether to accept or reject all elements together but it does
not attempt to take the posterior correlation between elements into
account. The `AdaptiveMetropolis` class (see below), on the other hand,
does make correlated proposals.

### DiscreteMetropolis

This class is just like `Metropolis`, but specialized to handle
`Stochastic` instances with dtype `int`. The jump proposal distribution
can either be `'Normal'`, `'Prior'` or `'Poisson'` (the default). In the
normal case, the proposed value is drawn from a normal distribution
centered at the current value and then rounded to the nearest integer.

### BinaryMetropolis

This class is specialized to handle `Stochastic` instances with dtype
`bool`.

For array-valued variables, `BinaryMetropolis` can be set to propose
from the prior by passing in `dist="Prior"`. Otherwise, the argument
`p_jump` of the init method specifies how probable a change is. Like
`Metropolis`' attribute `proposal_sd`, `p_jump` is tuned throughout the
sampling loop via `adaptive_scale_factor`.

### Automatic assignment of step methods

Every step method subclass (including user-defined ones) that does not
require any `__init__` arguments other than the stochastic variable to
be handled adds itself to a list called `StepMethodRegistry` in the PyMC
namespace. If a stochastic variable in an `MCMC` object has not been
explicitly assigned a step method, each class in `StepMethodRegistry` is
allowed to examine the variable.

To do so, each step method implements a class method called
`competence(stochastic)`, whose only argument is a single stochastic
variable. These methods return values from 0 to 3; 0 meaning the step
method cannot safely handle the variable and 3 meaning it will most
likely perform well for variables like this. The `MCMC` object assigns
the step method that returns the highest competence value to each of its
stochastic variables.

## Running MCMC Samplers

We can carry out Markov chain Monte Carlo sampling by calling the `sample` method (or in the terminal, `isample`) with the appropriate arguments.

In [ ]:
M = pm.MCMC(gelman_bioassay)
M.sample(10000, burn=5000)

In [ ]:
%matplotlib inline
pm.Matplot.plot(M.LD50)

## Imputation of Missing Data

As with most textbook examples, the models we have examined so far
assume that the associated data are complete. That is, there are no
missing values corresponding to any observations in the dataset.
However, many real-world datasets have missing observations, usually due
to some logistical problem during the data collection process. The
easiest way of dealing with observations that contain missing values is
simply to exclude them from the analysis. However, this results in loss
of information if an excluded observation contains valid values for
other quantities, and can bias results. An alternative is to impute the
missing values, based on information in the rest of the model.

For example, consider a survey dataset for some wildlife species:

    Count   Site   Observer   Temperature
    ------- ------ ---------- -------------
    15      1      1          15
    10      1      2          NA
    6       1      1          11

Each row contains the number of individuals seen during the survey,
along with three covariates: the site on which the survey was conducted,
the observer that collected the data, and the temperature during the
survey. If we are interested in modelling, say, population size as a
function of the count and the associated covariates, it is difficult to
accommodate the second observation because the temperature is missing
(perhaps the thermometer was broken that day). Ignoring this observation
will allow us to fit the model, but it wastes information that is
contained in the other covariates.

In a Bayesian modelling framework, missing data are accommodated simply
by treating them as unknown model parameters. Values for the missing
data $\tilde{y}$ are estimated naturally, using the posterior predictive
distribution:

$$p(\tilde{y}|y) = \int p(\tilde{y}|\theta) f(\theta|y) d\theta$$

This describes additional data $\tilde{y}$, which may either be
considered unobserved data or potential future observations. We can use
the posterior predictive distribution to model the likely values of
missing data.

Consider the coal mining disasters data introduced previously. Assume
that two years of data are missing from the time series; we indicate
this in the data array by the use of an arbitrary placeholder value,
`None`:

In [46]:
disasters_missing = np.array([ 4, 5, 4, 0, 1, 4, 3, 4, 0, 6, 3, 3, 4, 0, 2, 6,
3, 3, 5, 4, 5, 3, 1, 4, 4, 1, 5, 5, 3, 4, 2, 5,
2, 2, 3, 4, 2, 1, 3, -999, 2, 1, 1, 1, 1, 3, 0, 0,
1, 0, 1, 1, 0, 0, 3, 1, 0, 3, 2, 2, 0, 1, 1, 1,
0, 1, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 1, 0, 2,
3, 3, 1, -999, 2, 1, 1, 1, 1, 2, 4, 2, 0, 0, 1, 4,
0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1])

To estimate these values in PyMC, we generate a *masked array*. These are specialised NumPy arrays that contain a matching True or False value for each element to indicate if that value should be excluded from any computation. Masked arrays can be generated using NumPy's `ma.masked_equal` function:

In [47]:
disasters_masked = np.ma.masked_values(disasters_missing, value=-999)
disasters_masked

masked_array(data = [4 5 4 0 1 4 3 4 0 6 3 3 4 0 2 6 3 3 5 4 5 3 1 4 4 1 5 5 3 4 2 5 2 2 3 4 2
 1 3 -- 2 1 1 1 1 3 0 0 1 0 1 1 1 0 1 0 1 0 0 0 2 1 0 0 0 1 1 0 2 3 3 1 --
 2 1 1 1 1 2 4 2 0 0 1 4 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1],
             mask = [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False  True
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False],
       fill_value = -999)

This masked array, in turn, can then be passed to one of PyMC's data stochastic variables, which recognizes the masked array and replaces the missing values with stochastic variables of the desired type. For the coal mining disasters problem, recall that disaster events were modeled as Poisson variates:

```python
disasters = Poisson('disasters', mu=rate, observed=masked_values)
```

Each element in `disasters` is a Poisson random variable, irrespective of whether the observation was missing or not. The difference is that actual observations are assumed to be data stochastics, while the missing
values are unobserved stochastics. The latter are considered unknown, rather than fixed, and therefore estimated by the fitting algorithm, just as unknown model parameters are.

The entire model looks very similar to the original model:

In [61]:
with Model() as missing_data_model:

    # Prior for distribution of switchpoint location
    switchpoint = DiscreteUniform('switchpoint', lower=0, upper=len(disasters_masked))
    # Priors for pre- and post-switch mean number of disasters
    early_mean = Exponential('early_mean', lam=1.)
    late_mean = Exponential('late_mean', lam=1.)

    # Allocate appropriate Poisson rates to years before and after current
    # switchpoint location
    idx = np.arange(len(disasters_masked))
    rate = Deterministic('rate', switch(switchpoint >= idx, early_mean, late_mean))

    # Data likelihood
    disasters = Poisson('disasters', rate, observed=disasters_masked)

Applied log-transform to early_mean and added transformed early_mean_log to model.
Applied log-transform to late_mean and added transformed late_mean_log to model.


Here, we have used the `masked_array` function, rather than
`masked_equal`, and the value -999 as a placeholder for missing data.
The result is the same.

In [62]:
with missing_data_model:
    trace_missing = sample(2000)

Assigned Metropolis to switchpoint
Assigned NUTS to early_mean_log
Assigned NUTS to late_mean_log
Assigned Metropolis to disasters_missing
 [-----------------100%-----------------] 2000 of 2000 complete in 2.5 sec

In [63]:
missing_data_model.vars

[switchpoint, early_mean_log, late_mean_log, disasters_missing]

In [ ]:
from pymc3 import forestplot

forestplot(trace_missing, varnames=['disasters_missing'])

## References

1. M.I. Jordan. Graphical models. Statist. Sci., 19(1):140–155, 2004.